# Validacion 

En este documento se validan los resultados de FD-FEM y TD-FEM comparandolos con una solucion semi analitica.
Primero se definen unas dimensiones de recinto, posicion con posicion de receptor y emisor.


In [5]:
import  sys
import os 

dir_actual = os.path.abspath('')
dir_raiz = os.path.dirname(dir_actual)
# Añadir la ruta raíz al sys.path si aún no está
if dir_raiz not in sys.path:
    sys.path.append(dir_raiz)
print(f"Ruta raíz del proyecto añadida al path: {dir_raiz}")

Ruta raíz del proyecto añadida al path: /home/matias/Acoustic-Modal-Response-Optimizer


In [16]:
from mesh.mesh_3D_generator import generate_mesh_parallelepiped
#Frecuencia maxima de simulacion
f_max = 200
f_min = 20

#Dimensiones del Recinto
Lx = 3
Ly = 4
Lz = 2.5
L_floor = [Lx,Ly]

#Posicion de Emisor 
Sx = 2
Sy = 0.3
Sz = 1.5
Source_position = [Sx, Sy, Sz]

#Posicion de Receptor 
Rx = 1.5
Ry = 1.3
Rz = 1.2 
Receptor_position = [Sx, Sy, Sz]

#Generamos la malla
generate_mesh_parallelepiped(L_floor, Lz, Source_position, f_max)

Se guardó el volumen como 2                                                                                                                    
Límites encontrados para el volumen 2: [(2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 1)]
INFO: Se encontraron las 7 superficies esperadas.

--- Identificación de Superficies ---

--- Tags de Superficies Identificadas (Resultado Final) ---
Superficie 'superior': tag geométrico 4
Superficie 'inferior': tag geométrico 6
Superficie 'frontal': tag geométrico 7
Superficie 'trasera': tag geométrico 2
Superficie 'derecha': tag geométrico 5
Superficie 'izquierda': tag geométrico 3
Superficie 'fuente': tag geométrico 1

--- Asignando Grupos Físicos ---
Grupo Físico de Superficie 'superior' (Tag Físico: 1) asignado a la entidad geométrica 4.
Grupo Físico de Superficie 'inferior' (Tag Físico: 2) asignado a la entidad geométrica 6.
Grupo Físico de Superficie 'frontal' (Tag Físico: 3) asignado a la entidad geométrica 7.
Grupo Físico de Superficie 'tra

In [18]:
import numpy as np 
from matplotlib import pyplot as plt
from FEM.FEM_source_time import FEM_time_grid 
from FEM.FEM_source import FEM_Source_Solver
from FEM.mode_sumation import compute_modal_transfer
path = r"notebooks/mallado/room.msh"
td_rir, frecs = FEM_time_grid(path, Receptor_position, f_min, f_max)
td_rir = td_rir[0] #descarto las posiciones extra de la grilla 

plt.figure()
plt.plot(frecs, td_rir)

plt.show()



TypeError: FEM_time_grid() missing 1 required positional argument: 'h_minm'